In [5]:
import InfoAPI
import requests
import json

keys = InfoAPI.InfoAPI()

In [13]:
def main(a, b):
    """Gets travel summary"""
    if len(a) or len(b):
        search = a
        search_2 = b
    else:
        search = input("Address 1\n>>>> ")
        search_2 = input("Address 2\n>>>> ")
        
    lat, long, lat_2, long_2 = get_geo(search, search_2)
    return get_route(lat, long, lat_2, long_2)

In [3]:
def get_geo(search_text_1, search_text_2):
    """Calculates route
    
    Parameters
    ----------
    search_text_1 : string
        Address in format [number street, city, state zipcode]
        
    search_text_2 : string
        Address in format [number street, city, state zipcode]
        
    Returns
    -------
    lat : float
        First geographic latitude location.
        
    long : float
        First geographic longitude location.
        
    lat_2 : float
        Second geographic latitude location.
        
    long_2 : float
        Second geographic longitude location.
        
    """
    
    BASE_GEO = "https://geocoder.api.here.com/6.2/geocode.json"
    
    params_geo = {
        "app_id": keys.APP_ID,
        "app_code": keys.APP_CODE,
        "searchtext": search_text_1,
    }
    
    req = requests.get(BASE_GEO, params=params_geo)
    info = req.json()["Response"]

    lat, long = info['View'][0]["Result"][0]["Location"]["DisplayPosition"].values()
    
    params_geo['searchtext'] = search_text_2
    
    req = requests.get(BASE_GEO, params=params_geo)
    info = req.json()["Response"]

    lat_2, long_2 = info['View'][0]["Result"][0]["Location"]["DisplayPosition"].values()
    
    return (lat, long, lat_2, long_2)

In [7]:
def get_route(lat, long, lat_2, long_2):
    """Calculates route
    
    Parameters
    ----------
    lat : float
        First geographic latitude location.
        
    long : float
        First geographic longitude location.
        
    lat_2 : float
        Second geographic latitude location.
        
    long_2 : float
        Second geographic longitude location.
        
    Returns
    -------
    total_time : float
        In minutes.
    
    traffic_time : float
        In minutes.
        
    traffic_percent : float
        
        
    flags : list
    
    """
    BASE_ROUTING = "https://route.api.here.com/routing/7.2/calculateroute.json"
    
    params_routing = {
    "app_id": keys.APP_ID,
    "app_code": keys.APP_CODE,
    "waypoint0": "geo!" + str(lat) + "," + str(long),
    "waypoint1": "geo!" + str(lat_2) + "," + str(long_2),
    "mode": "fastest;car;traffic:enabled",
    }
    
    req = requests.get(BASE_ROUTING, params=params_routing)

    info = req.json()['response']
    summary = info['route'][0]['summary']

    total_time = summary['travelTime'] / 60
    traffic_time = (summary['trafficTime'] - summary['baseTime']) / 60
    traffic_percent = traffic_time / summary['baseTime']
    flags = summary['flags']
    
    return {"Total Time": total_time,
            "Traffic Time": traffic_time,
            "Traffic Percent": traffic_percent,
            "Flags": flags}

In [14]:
results = main("Killeen, TX", "Austin, TX")

In [16]:
print(results)

{'Total Time': 86.55, 'Traffic Time': 3.55, 'Traffic Percent': 0.0007128514056224899, 'Flags': ['tollroad', 'motorway', 'builtUpArea']}
